In [9]:
!pip install sentence-transformers

In [8]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from sentence_transformers import SentenceTransformer
import ast
import faiss
# import json

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')

# 1) Load to Memory & Merge Features and Desc.

In [ ]:
games_df = pd.read_csv('games.csv')
metadata_df = pd.read_json('games_metadata.json', lines=True)

df = pd.merge(games_df, metadata_df, on='app_id', how='inner')

games_df = None
metadata_df = None

In [ ]:
# df.to_csv('Altered CSVs/merged_game_data.csv', index=False)

In [ ]:
# df = pd.read_csv('Altered CSVs/merged_game_data.csv')

# 2.2) MultiLabelBinarizer

In [ ]:
# df = pd.read_csv('Altered CSVs/merged_game_data.csv')

In [ ]:
mlb = MultiLabelBinarizer()

# Convert the 'tags' column from a string representation of a list to an actual list
df['tags'] = df['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

tag_matrix = mlb.fit_transform(df['tags'])

tags_df = pd.DataFrame(tag_matrix, columns=mlb.classes_)

df = pd.concat([df, tags_df], axis=1)

In [ ]:
# df.to_csv('Altered CSVs/MutliLabelBinarized.csv', index=False)

# 3.2) Sentance Transformer / Embeddings



In [ ]:
# df = pd.read_csv('Altered CSVs/MutliLabelBinarized.csv')

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

df['title'] = df['title'].fillna('')
df['description'] = df['description'].fillna('')

df['text_for_embedding'] = df['title'] + ' ' + df['description'] + ' ' + df['tags']

df['embedding'] = list(text_embeddings)

In [ ]:
# df.to_csv('Altered CSVs/with_embeddings.csv', index=False)

# 4.2) Normalization

## Min-Max

In [ ]:
# df = pd.read_csv('Altered CSVs/with_embeddings.csv')

**Applied for Ratings**

In [ ]:
rating_dict = {
    'Overwhelmingly Positive': 9,
    'Very Positive': 8,
    'Positive': 7,
    'Mostly Positive': 6,
    'Mixed': 5,
    'Mostly Negative': 4,
    'Negative': 3,
    'Very Negative': 2,
    'Overwhelmingly Negative': 1
}

df['rating_normalized'] = (df['rating'].map(rating_dict) - 1) / 8

## Log Normalizing

**Applied for Positive Ratio, Price, and Review Count**

In [11]:
df['positive_ratio_log'] = np.log1p(df['positive_ratio'])
df['price_log'] = np.log1p(df['price_final'])
df['user_reviews_log'] = np.log1p(df['user_reviews'])

In [ ]:
# df.to_csv('Altered CSVs/normalized.csv', index=False)

# 5.2) Concatenating Features

In [ ]:
# df = pd.read_csv('Altered CSVs/normalized.csv')

In [ ]:
numerics = df[['price_log', 'positive_ratio_log', 'user_reviews_log']]

X = np.hstack([numerics])